In [1]:
#!pip install openai

# gcloud config set project tourguide-388723
# gcloud auth login
# gcloud auth activate-service-account 

In [17]:
22908221943 / 1000 * 0.00025

5727.05548575

In [2]:
import os
import sys

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/mcmenamin/Repos/tourguide/creds/tourguide-388723-45d84f790d24.json"

In [4]:
sys.path.insert(0, '/Users/mcmenamin/Repos/tourguide/')  # Add the directory containing the module to the Python path
PROJECT_ID = "tourguide-388723"

In [5]:
import json
from google.cloud import bigquery

from flaskapp.bigquery_interface import BigQueryInterface

with open("../creds/open_ai.json", "rt") as file:
    openai_creds = json.load(file)

In [6]:
def lookup_fulltext(title):
    with BigQueryInterface(project_id=PROJECT_ID) as bq:
        query_str = f"SELECT page_id, title, raw_text FROM pages.links WHERE title = '{title}'"
        df_result = bq.query(query=query_str)
    return df_result.loc[0, "raw_text"]

In [7]:
OPENAI_ORG = openai_creds["org"]
OPENAI_API_KEY = openai_creds["api_key"]

In [8]:
from openai import OpenAI

client = OpenAI(organization=OPENAI_ORG, api_key=OPENAI_API_KEY)

In [15]:
title = "Carson Park (Eau Claire, Wisconsin)"
#title = "Parker 13-Sided Barn"
title = "Austin, Texas"
full_text = lookup_fulltext(title)

In [16]:
full_text

'\n\n\n\n\n\n\n\'\'\'Austin\'\'\' ( ,  ) is the [[List of capitals in the United States|capital]] of the [[U.S. state]] of [[Texas]], as well as the [[county seat|seat]] and most populous city of [[Travis County, Texas|Travis County]], with portions extending into [[Hays County, Texas|Hays]] and [[Williamson County, Texas|Williamson]] counties. Incorporated on December 27, 1839, it is the [[List of United States cities by population|10th most populous city]] in the United States, the [[List of cities in Texas by population|fourth most populous city]] in the state after [[Dallas]], [[San Antonio]] and [[Houston]], and the second most populous state capital city after [[Phoenix, Arizona|Phoenix]], the capital of [[Arizona]]. It has been one of the fastest growing large cities in the United States since 2010.   [[Downtown Austin]] and [[Downtown San Antonio]] are approximately 80. mi apart, and both fall along the [[Interstate 35]] corridor. This combined metropolitan region of [[San Anto

In [10]:
import itertools
first_paras = itertools.islice(
    (p for p in full_text.strip().split("\n") if p),
    2
)
beginning_text = "\n\n".join(first_paras)

In [11]:
print(beginning_text)

'''Austin''' ( ,  ) is the [[List of capitals in the United States|capital]] of the [[U.S. state]] of [[Texas]], as well as the [[county seat|seat]] and most populous city of [[Travis County, Texas|Travis County]], with portions extending into [[Hays County, Texas|Hays]] and [[Williamson County, Texas|Williamson]] counties. Incorporated on December 27, 1839, it is the [[List of United States cities by population|10th most populous city]] in the United States, the [[List of cities in Texas by population|fourth most populous city]] in the state after [[Dallas]], [[San Antonio]] and [[Houston]], and the second most populous state capital city after [[Phoenix, Arizona|Phoenix]], the capital of [[Arizona]]. It has been one of the fastest growing large cities in the United States since 2010.   [[Downtown Austin]] and [[Downtown San Antonio]] are approximately 80. mi apart, and both fall along the [[Interstate 35]] corridor. This combined metropolitan region of [[San Antonio–Austin metroplex|

In [12]:
model_params = {
    "model":  "gpt-3.5-turbo-0125",  # "gpt-4",  # 
    "temperature": 0.1,
    "n": 1,
    "stream": False,
    "response_format": {"type": "json_object"}
}

job_prompt = """
Create a short sentence to describe the location of the thing in this article, and provide a list of geographic locations in the summary.
Each location must correspond to a linked page in the original article.
In the original article, links are identified by being wrapped in double square brackets, like this: `[[link]]`.
The output should be formatted as JSON like {"summary": "summary text", "locations": ["place1", "place2", ...]}.
""".strip()


response = client.chat.completions.create(
    **model_params,
    messages=[
    {
        "role": "system",
        "content": job_prompt,
        "name": "place_finder"
    },
    {
        "role": "user",
        "content": beginning_text,
    }
    ]
)

In [108]:
lower_text = beginning_text.lower()
for choice in response.choices:
    data = json.loads(choice.message.content)
    print(data)


    linked_locations = [
        l
        for l in data["locations"]
        if (
            f"[[{l.lower()}]]" in lower_text
            or f"[[{l.lower()}|" in lower_text
        )
    ]

    print("\n" + "\n".join(linked_locations))
print(f"{response.usage.total_tokens} tokens")

{'summary': 'Austin is the capital of Texas, located in Travis County with extensions into Hays and Williamson counties. It is the 10th most populous city in the U.S. and the fourth most populous city in Texas, situated along the Interstate 35 corridor. The San Antonio–Austin metroplex has approximately 5 million people. Austin is a Beta-level global city and the cultural and economic center of the Austin–Round Rock metropolitan statistical area.', 'locations': ['Austin', 'Texas', 'Travis County', 'Hays County', 'Williamson County', 'Dallas', 'San Antonio', 'Houston', 'Phoenix, Arizona', 'Downtown Austin', 'Downtown San Antonio', 'Interstate 35', 'San Antonio–Austin metroplex', 'contiguous United States', 'Lady Bird Lake', 'Lake Travis', 'Colorado River (Texas)', 'Barton Springs', 'McKinney Falls State Park', 'Lake Walter E. Long', 'Texas Hill Country']}

Texas
Dallas
San Antonio
Houston
Phoenix, Arizona
Downtown Austin
Downtown San Antonio
Interstate 35
San Antonio–Austin metroplex
co